In [5]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

In [6]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.9_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.9_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 32

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

Found 1112569 images belonging to 5270 classes.
Found 119010 images belonging to 5270 classes.


In [7]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'

# Checkpointer 
checkpointer = ModelCheckpoint(filepath='saved_models/Resnet50.{epoch:02d}-{loss:.2f}-{val_loss:.2f}_RandomSplit_Train_0.9_0.1.h5', verbose=1)

# create the base pre-trained model
base_model = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions, name= 'Resenet50')

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

num_epochs = 5


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])


Epoch 1/5
34767/34767 [==============================] - ETA: 0s - loss: 4.9659 - acc: 0.2334 - val_loss: 4.8503 - val_acc: 0.2331
Epoch 2/5
34767/34767 [==============================] - ETA: 0s - loss: 3.9990 - acc: 0.3476 - val_loss: 4.1987 - val_acc: 0.3163
Epoch 3/5
34767/34767 [==============================] - ETA: 0s - loss: 3.6858 - acc: 0.3857 - val_loss: 4.0679 - val_acc: 0.3288
Epoch 4/5
34767/34767 [==============================] - ETA: 0s - loss: 3.4975 - acc: 0.4086 - val_loss: 5.2937 - val_acc: 0.1908
Epoch 5/5
34767/34767 [==============================] - ETA: 0s - loss: 3.3666 - acc: 0.4235 - val_loss: 3.7671 - val_acc: 0.3725


In [8]:
num_epochs = 2


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])

Epoch 1/2
34767/34767 [==============================] - ETA: 0s - loss: 3.2651 - acc: 0.4350 - val_loss: 4.2240 - val_acc: 0.3103
Epoch 2/2
34767/34767 [==============================] - ETA: 0s - loss: 3.1835 - acc: 0.4438 - val_loss: 3.7979 - val_acc: 0.3880
